In [2]:
from ppo_model import *
from utils import *

In [3]:
N_PLAYER = 3
device = 'cuda' if torch.cuda.is_available() else 'cpu'

path = f'./ppo_weight/model_gen_5_default_rwd_57_iter.pth'
if 'gen_2' in path:
    model = ppo_gen_2(N_PLAYER).to(device)
elif 'gen_3' in path:
    model = ppo_gen_3(N_PLAYER).to(device)
    if 'gen_3_5' in path:
        model.gen = 3.5
elif 'gen_4' in path:
    model = ppo_gen_4(N_PLAYER).to(device)
elif 'gen_5' in path:
    model = ppo_gen_5(N_PLAYER).to(device)
    if 'gen_5_5' in path:
        model.gen = 5.5   

model.load_state_dict(torch.load(path,  map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
rank_dict = {
    0: 3,
    1: 2,
    2: 1
}


In [ ]:
n_game = 1000
n_win = 0
rank_list = []

for _ in range(n_game):
    nothanks = game()
    ppo_turn = random.choice([0,1,2])
    while nothanks.is_continue:
        # print('------------------------------')
        # print(f'''Card: {nothanks.current_card} | Chip in pot: {nothanks.chip_in_pot} | Player: {nothanks.turn} - {nothanks.players[nothanks.turn]}\n''')
        # print('------------------------------')
        
        if nothanks.turn == ppo_turn:
            with torch.no_grad():
                if model.gen == 2:
                    current_state = torch.tensor(nothanks.encode_state_gen_2()).to(device)
                elif model.gen == 3:
                    current_state = torch.tensor(nothanks.encode_state_gen_3(nothanks.get_state)).to(device)
                elif model.gen == 3.5:
                    current_state = torch.tensor(nothanks.encode_state_gen_3(nothanks.get_state_gen_3_5)).to(device)
                elif model.gen == 4:
                    current_state = torch.tensor(nothanks.encode_state_gen_3(nothanks.get_state_gen_4)).to(device)
                elif model.gen == 5:
                    x_card, x_state = nothanks.encode_state_gen_5(nothanks.get_state)
                    x_card = torch.tensor(x_card).float().unsqueeze(1)
                    x_state = torch.tensor(x_state)
                elif model.gen == 5.5:
                    x_card, x_state = nothanks.encode_state_gen_5(nothanks.get_state_gen_3_5)
                    x_card = torch.tensor(x_card).float().unsqueeze(1)
                    x_state = torch.tensor(x_state)
                    
                legal_move = nothanks.get_legal_action() # a list 
                legal_move_mask = torch.tensor([False if move in legal_move else True for move in nothanks.move_encode.values()]).to(device)
                if model.gen in [5, 5.5]:
                    move_raw, log_prob, entropy, value = model.forward(x_card, x_state, legal_move_mask)
                else:
                    move_raw, log_prob, entropy, value = model.forward(current_state, legal_move_mask)
                move = nothanks.move_encode.get(move_raw.item())
        else :
            move = nothanks.rollout_policy_3(p = 0.9)
            # move = nothanks.rollout_policy_rule()

        # print(f"""Move taken: {move}\n""")
        nothanks.action(move)
        # game_node = game_state(turn = nothanks.turn)

    game_result = nothanks.eval_rank()
    winner_index = np.argmax(game_result)
    
    rank_list.append(rank_dict.get(game_result[ppo_turn]))
    if winner_index == ppo_turn:
        n_win += 1

In [15]:
print(n_win/n_game)

0.683


In [17]:
np.mean(rank_list)

np.float64(1.348)